In [1]:
import numpy as np
import pandas as pd
import torch.nn as nn
import torch
import os
import random

from functions.parse_data import synth_dataloader
from multivariate_quantile_regression.network_model import QuantileNetwork

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from cot_train.utils import MLP5

/tmp/ipykernel_277949/3275681278.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-03-25 13:16:19.933507: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-25 13:16:19.967350: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-25 13:1

In [2]:
# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    # CUDA is available, so let's set default device to GPU
    torch.set_default_device(0)
    print("CUDA is available. Using GPU.")
else:
    # CUDA is not available, so let's use the CPU
    print("CUDA is not available. Using CPU.")

# Example usage:
tensor = torch.randn(3, 3)  # Create a tensor on the selected device
print("Tensor is on device:", tensor.device)
device = tensor.device

CUDA is available. Using GPU.


Tensor is on device: cuda:0


In [3]:
#Load data and inspect
df = synth_dataloader('SMHIdata')
df.head(10)

,Cloud_B02,Cloud_B03,Cloud_B04,Cloud_B05,Cloud_B06,Cloud_B07,Cloud_B08,Cloud_B08A,Cloud_B09,Cloud_B10,...,Clear_B11,Clear_B12,Sat_Zenith_Angle,Sun_Zenith_Angle,Azimuth_Diff_Angle,COT,Cloud_Type,Profile_ID,GOT,Water_Vapor
0,0.94195,0.87799,0.92936,0.93407,0.95181,0.96217,0.92871,0.97181,0.49957,0.04136,...,0.12946,0.18888,4.53,52.05,167.66,5.897,3,3335,0.126,0.35
1,0.30422,0.40100,0.27834,0.57800,1.01964,1.02787,1.00519,1.03599,0.59139,0.01055,...,0.71532,0.36823,12.85,41.68,161.91,1.275,2,1996,0.126,0.31
2,0.28715,0.25066,0.30366,0.29214,0.34088,0.40079,0.37376,0.48750,0.02092,0.00067,...,0.86232,0.63915,14.53,79.23,168.52,1.799,1,6796,0.127,4.04
3,0.27146,0.33719,0.19841,0.46411,0.88787,0.89584,0.87746,0.90439,0.51811,0.00561,...,0.56307,0.23663,6.54,70.23,165.49,0.519,2,3701,0.123,0.22
4,0.39689,0.38594,0.32623,0.37338,0.60678,0.66895,0.55343,0.70168,0.01513,0.00049,...,0.56472,0.20853,8.56,75.15,148.48,8.569,2,6345,0.128,5.40
5,0.75592,0.65853,0.70670,0.71369,0.73147,0.74182,0.71190,0.75003,0.32287,0.00328,...,0.92187,0.87515,12.38,73.72,153.05,16.874,3,1419,0.126,0.51
6,0.34660,0.30719,0.27459,0.35268,0.72965,0.74705,0.69842,0.75893,0.18345,0.00089,...,0.64499,0.27212,14.74,73.05,13.63,3.589,3,424,0.122,0.99
7,0.22494,0.33843,0.19615,0.53821,0.98347,0.99256,0.96757,1.00132,0.53669,0.00511,...,0.61388,0.28605,8.24,53.20,146.76,0.624,2,3427,0.125,0.35
8,0.57982,0.62369,0.57909,0.75147,0.99608,1.04055,0.94017,1.07212,0.43014,0.03328,...,0.94942,0.49475,11.57,38.00,117.48,13.909,1,6884,0.122,1.88
9,0.83664,0.80620,0.81432,0.78581,0.81063,0.84108,0.78542,0.85153,0.26374,0.00122,...,0.90206,0.89850,9.69,45.83,23.04,24.937,3,7455,0.106,1.77


Case 1: Exclude COT estimation in input

In [4]:
#Set columns for X and y (input/output features)
X_cols = ['Cloud_B02','Cloud_B03','Cloud_B04','Cloud_B05','Cloud_B06',
          'Cloud_B07','Cloud_B08','Cloud_B08A','Cloud_B09','Cloud_B10','Cloud_B11','Cloud_B12','Sun_Zenith_Angle']
y_cols = ['Clear_B02','Clear_B03','Clear_B04','Clear_B05','Clear_B06',
          'Clear_B07','Clear_B08','Clear_B08A','Clear_B09','Clear_B10','Clear_B11','Clear_B12']

#Find X and y
X=df[X_cols]
y=df[y_cols]

#Separate testdata from rest for 80/10/10 Train/Val/Test split
X_trainval, X_test, y_trainval, y_test=train_test_split(X,y,test_size=0.1,random_state=313)

#Add noise to X_test, 0 mean with stdev equal to 3% of mean of each feature
np.random.seed(313)
X_test = X_test + np.random.randn(np.shape(X_test)[0],np.shape(X_test)[1]) * np.mean(X.to_numpy(),axis=0)*0.03

In [5]:
#Set up which quantiles to estimate, and find index of estimator (q=0.5)
quantiles=np.array([0.1,0.5,0.9])
est= np.where(quantiles==0.5)[0].item()

#Set up algorithm parameters for both cases
val_size=0.1
num_models=5 #Set number of models in ensemble
batch_size=500
nepochs=1000
lr=0.003
noise_ratio = 0.03
early_break=True

In [6]:
#Choose if to save models and metrics, if so set path
save = True
if save:
    test_name = "COT_est_inp_noCOT"
    main_filepath = 'pytorch_models/'+test_name

#Set up NN structure
no_nodes = 100

sequence= lambda: nn.Sequential(
    nn.Linear(len(X_cols),no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes, len(quantiles)*len(y_cols)) #Output dimesion is number of quantiles times number of target variables
)

#Initalize models
models = [QuantileNetwork(quantiles=quantiles) for _ in range(num_models)]

#Train models
for i,model in enumerate(models):
    #Find new train/val splits for each model for robustness
    validation_indices=np.array(random.sample(range(len(X_trainval['Cloud_B02'])), int(len(X['Cloud_B02'])*val_size)))
    train_indices=[i for i in range(len(X_trainval['Cloud_B02'])) if np.any(validation_indices==i)==False]  
    #Fit model
    model.fit(X_trainval.to_numpy(),y_trainval.to_numpy(), 
            train_indices=train_indices, 
            validation_indices=validation_indices, 
            batch_size=batch_size,
            nepochs=nepochs,
            sequence=sequence(),
            lr=lr,
            noise_ratio=noise_ratio,
            early_break=early_break)
    
    #Save models if wanted
    if save:
        filepath=main_filepath+'/model'+str(i)
        os.makedirs(filepath,exist_ok=True)
        torch.save(model,filepath+'/model_file')
    

Epoch 401


Batch number: 100%|██████████| 320/320 [00:01<00:00, 270.31it/s]

Training loss [2.096641] Validation loss [2.1193802]


---No improvement in 100 epochs, broke early---
Best model out of total max epochs found at epoch 301
With validation loss: 2.0930285453796387


In [7]:
#Initialize dataframe for error metrics and array for ensemble predictions
noCOT_model_metrics=pd.DataFrame(columns=['Ensemble_mean','Ensemble_index','MSE','PSNR','R2_score','Mean_Quantile_Loss'])
preds_total=[]
#Make predictions and evaluate
for i,model in enumerate(models):
    preds = model.predict(X_test.to_numpy())
    #Keep track of ensemble prediction
    if i==0:
        preds_total=preds
    else:
        preds_total=preds_total+preds

    #Find errors
    mse=mean_squared_error(y_test.to_numpy(),preds[:,:,est])
    psnr=QuantileNetwork.PSNR(y_test.to_numpy(),preds[:,:,est])
    r2=r2_score(y_test.to_numpy(),preds[:,:,est])
    mean_quantile=QuantileNetwork.mean_marginal_loss(y_test.to_numpy(),preds,quantiles)
    #Add to dataframe
    tmp_metrics=pd.DataFrame(data=[[False,i,mse,psnr,r2,mean_quantile]],columns=['Ensemble_mean','Ensemble_index','MSE','PSNR','R2_score','Mean_Quantile_Loss'])
    noCOT_model_metrics=pd.concat([noCOT_model_metrics,tmp_metrics])


#Now do the same for ensemble predictions
preds_total=preds_total/num_models

mse=mean_squared_error(y_test.to_numpy(),preds_total[:,:,est])
psnr=QuantileNetwork.PSNR(y_test.to_numpy(),preds_total[:,:,est])
r2=r2_score(y_test.to_numpy(),preds_total[:,:,est])
mean_quantile=QuantileNetwork.mean_marginal_loss(y_test.to_numpy(),preds_total,quantiles)

tmp_metrics=pd.DataFrame(data=[[True,np.nan,mse,psnr,r2,mean_quantile]],columns=['Ensemble_mean','Ensemble_index','MSE','PSNR','R2_score','Mean_Quantile_Loss'])
noCOT_model_metrics=pd.concat([noCOT_model_metrics,tmp_metrics])

#Save metrics if we want to
if save:
    noCOT_model_metrics=noCOT_model_metrics.reset_index(drop=True)
    noCOT_model_metrics.to_csv(main_filepath+'/model_metrics.csv',index=False)
    

/tmp/ipykernel_277949/1313021597.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  noCOT_model_metrics=pd.concat([noCOT_model_metrics,tmp_metrics])


Case 2: Use Aleksis COT estimation as input to model

In [8]:
#Set up paths for importing COT est models
COT_model_paths = ['smhi_models3/0/model_it_2000000','smhi_models3/1/model_it_2000000','smhi_models3/2/model_it_2000000','smhi_models3/3/model_it_2000000','smhi_models3/4/model_it_2000000']

#Initialize and load COT estimation models
COT_est_models = [MLP5(13, 1, apply_relu=True) for _ in range(len(COT_model_paths))]
for i,model in enumerate(COT_est_models):
    model.load_state_dict(torch.load(COT_model_paths[i],map_location=device))

#Create X for COT estimation (no angles)
X_COTest = X.to_numpy()
#Add noise for fairness
X_COTest = X_COTest + np.random.randn(np.shape(X_COTest)[0],np.shape(X_COTest)[1]) * np.mean(X_COTest,axis=0)*0.03
#Normalize and turn into tensor before input
X_COTest_mu = np.mean(X_COTest,axis=0)
X_COTest_std = np.std(X_COTest,axis=0)
X_COTest_norm = (X_COTest-X_COTest_mu)/X_COTest_std
tX_COTest_norm = torch.Tensor(X_COTest_norm).to(device)
#Make predictions (*50 to denormalize predictions)
COT_preds_total = []
for i,model in enumerate(COT_est_models):
    COT_preds = 50*model(tX_COTest_norm).cpu().detach().numpy()
    #Keep track of ensemble prediction
    if i==0:
        COT_preds_total=COT_preds
    else:
        COT_preds_total=COT_preds_total+COT_preds

COT_preds_total = COT_preds_total/len(COT_est_models)

#Now separate into trainval and test
COT_preds_total_trainval = COT_preds_total[X_trainval.index,0]
COT_preds_total_test = COT_preds_total[X_test.index,0]

#Create new X's including COT estimation
X_trainval_COT = X_trainval.assign(COT_est=COT_preds_total_trainval)
X_test_COT = X_test.assign(COT_est=COT_preds_total_test)

In [9]:
#Choose if to save models and metrics, if so set path
save = True
if save:
    test_name = "COT_est_inp_wCOT"
    main_filepath = 'pytorch_models/'+test_name

#Create new net with 1 additional input
sequence= lambda: nn.Sequential(
    nn.Linear(len(X_cols)+1,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes, len(quantiles)*len(y_cols)) #Output dimesion is number of quantiles times number of target variables
)

#Initalize models
models = [QuantileNetwork(quantiles=quantiles) for _ in range(num_models)]

#Train models
for i,model in enumerate(models):
    #Find new train/val splits for each model for robustness
    validation_indices=np.array(random.sample(range(len(X_trainval['Cloud_B02'])), int(len(X['Cloud_B02'])*val_size)))
    train_indices=[i for i in range(len(X_trainval['Cloud_B02'])) if np.any(validation_indices==i)==False]  
    #Fit model with X including COT_est
    model.fit(X_trainval_COT.to_numpy(),y_trainval.to_numpy(), 
            train_indices=train_indices, 
            validation_indices=validation_indices, 
            batch_size=batch_size,
            nepochs=nepochs,
            sequence=sequence(),
            lr=lr,
            noise_ratio=noise_ratio,
            early_break=early_break)
    
    #Save models if wanted
    if save:
        filepath=main_filepath+'/model'+str(i)
        os.makedirs(filepath,exist_ok=True)
        torch.save(model,filepath+'/model_file')
    

Epoch 441


Batch number: 100%|██████████| 320/320 [00:00<00:00, 354.91it/s]


Training loss [2.052324] Validation loss [2.103987]
Epoch 442


Batch number: 100%|██████████| 320/320 [00:00<00:00, 366.04it/s]

Training loss [2.0548227] Validation loss [2.111849]
Epoch 443


...

Training loss [2.0519552] Validation loss [2.081174]
Epoch 448


Batch number: 100%|██████████| 320/320 [00:00<00:00, 365.91it/s]


Training loss [2.0474522] Validation loss [2.092672]
Epoch 449


Batch number: 100%|██████████| 320/320 [00:00<00:00, 365.47it/s]


Training loss [2.0490413] Validation loss [2.107248]
---No improvement in 100 epochs, broke early---
Best model out of total max epochs found at epoch 349
With validation loss: 2.0767927169799805


In [10]:
#Initialize dataframe for error metrics and array for ensemble predictions
wCOT_model_metrics=pd.DataFrame(columns=['Ensemble_mean','Ensemble_index','MSE','PSNR','R2_score','Mean_Quantile_Loss'])
preds_total=[]
#Make predictions and evaluate
for i,model in enumerate(models):
    preds = model.predict(X_test_COT.to_numpy())
    #Keep track of ensemble prediction
    if i==0:
        preds_total=preds
    else:
        preds_total=preds_total+preds

    #Find errors
    mse=mean_squared_error(y_test.to_numpy(),preds[:,:,est])
    psnr=QuantileNetwork.PSNR(y_test.to_numpy(),preds[:,:,est])
    r2=r2_score(y_test.to_numpy(),preds[:,:,est])
    mean_quantile=QuantileNetwork.mean_marginal_loss(y_test.to_numpy(),preds,quantiles)
    #Add to dataframe
    tmp_metrics=pd.DataFrame(data=[[False,i,mse,psnr,r2,mean_quantile]],columns=['Ensemble_mean','Ensemble_index','MSE','PSNR','R2_score','Mean_Quantile_Loss'])
    wCOT_model_metrics=pd.concat([wCOT_model_metrics,tmp_metrics])


#Now do the same for ensemble predictions
preds_total=preds_total/num_models

mse=mean_squared_error(y_test.to_numpy(),preds_total[:,:,est])
psnr=QuantileNetwork.PSNR(y_test.to_numpy(),preds_total[:,:,est])
r2=r2_score(y_test.to_numpy(),preds_total[:,:,est])
mean_quantile=QuantileNetwork.mean_marginal_loss(y_test.to_numpy(),preds_total,quantiles)

tmp_metrics=pd.DataFrame(data=[[True,np.nan,mse,psnr,r2,mean_quantile]],columns=['Ensemble_mean','Ensemble_index','MSE','PSNR','R2_score','Mean_Quantile_Loss'])
wCOT_model_metrics=pd.concat([wCOT_model_metrics,tmp_metrics])

#Save metrics if we want to
if save:
    wCOT_model_metrics=wCOT_model_metrics.reset_index(drop=True)
    wCOT_model_metrics.to_csv(main_filepath+'/model_metrics.csv',index=False)

/tmp/ipykernel_277949/3770301511.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  wCOT_model_metrics=pd.concat([wCOT_model_metrics,tmp_metrics])


In [11]:
#Display noCOT results
noCOT_model_metrics

,Ensemble_mean,Ensemble_index,MSE,PSNR,R2_score,Mean_Quantile_Loss
0,False,0,0.006720,21.755291,0.847775,0.434261
1,False,1,0.006769,21.723755,0.846973,0.435420
2,False,2,0.006786,21.712573,0.846030,0.434844
3,False,3,0.006757,21.730852,0.846827,0.434545
4,False,4,0.006922,21.626183,0.842699,0.435957
5,True,NaN,0.006402,21.965449,0.854970,0.416566


In [12]:
#Display wCOT results
wCOT_model_metrics

,Ensemble_mean,Ensemble_index,MSE,PSNR,R2_score,Mean_Quantile_Loss
0,False,0,0.006758,21.730404,0.847425,0.433127
1,False,1,0.006709,21.762078,0.848158,0.432052
2,False,2,0.006690,21.774598,0.848998,0.430125
3,False,3,0.006662,21.792717,0.849113,0.429059
4,False,4,0.006684,21.778116,0.848828,0.429949
5,True,NaN,0.006339,22.008468,0.856733,0.412773
